In [1]:
'''
The code is used to train BC imitator, or pretrained GAIL imitator
'''

import argparse
import tempfile
import os.path as osp
import gym
import logging
from tqdm import tqdm

import tensorflow as tf

from baselines.gail import mlp_policy
from baselines import bench
from baselines import logger
from baselines.common import set_global_seeds, tf_util as U
from baselines.common.misc_util import boolean_flag

#from baselines.gail.run_mujoco import runner
from baselines.gail.dataset.mujoco_dset import Mujoco_Dset 

##from baselines.common.mpi_adam import MpiAdam

In [2]:
from gfrl.base.run_my_ppo2 import create_single_scenic_environment
network = "gfootball_impala_cnn"
scenario = "../_scenarios/academy/pass_n_shoot.scenic"

#load_path = "/home/ubuntu/ScenicGFootBall/training/gfrl/_saved_models/pass_shoot_5M"
load_path = "/home/ubuntu/ScenicGFootBall/training/gfrl/_res/pass_n_shoot_3/checkpoints/final_00610"



In [3]:


import tensorflow.compat.v1 as tf
import multiprocessing

from baselines.common.vec_env.dummy_vec_env import DummyVecEnv
vec_env = DummyVecEnv([lambda _i=i: \
                        create_single_scenic_environment(_i,scenario) for i in
                        range(1)])



ncpu = multiprocessing.cpu_count()
config = tf.ConfigProto(allow_soft_placement=True,
                        intra_op_parallelism_threads=ncpu,
                        inter_op_parallelism_threads=ncpu)
config.gpu_options.allow_growth = True
tf.Session(config=config).__enter__()

from gfrl.common.mybase import ppo2
model = ppo2.learn(
    network=network,
    total_timesteps=0,
    env=vec_env,
    nsteps=512,
    load_path=load_path
    )




/home/ubuntu/.local/lib/python3.7/site-packages/scenic/core/errors.py:160: UserWarning: unable to install sys.excepthook to format Scenic backtraces
  warnings.warn('unable to install sys.excepthook to format Scenic backtraces')


Scenic Environment:  ../_scenarios/academy/pass_n_shoot.scenic
pygame 2.0.1 (SDL 2.0.14, Python 3.7.10)
Hello from the pygame community. https://www.pygame.org/contribute.html
Logging to /tmp/openai-2021-04-27-20-12-08-152292





Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Please use `layer.__call__` method instead.





Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where






In [4]:
from gfrl.dm.my_runner import MyTrajRunner

from baselines.common.vec_env.dummy_vec_env import DummyVecEnv
vec_env = DummyVecEnv([lambda _i=i: \
                        create_single_scenic_environment(_i,scenario) for i in
                        range(1)])

num_episodes = 500
runner = MyTrajRunner(env=vec_env, model=model, nsteps=512, num_episodes=num_episodes)
mb_obs, mb_rewards, mb_dones, mb_actions, mb_values, mb_neglogpacs, mb_states, epinfos, mb_gt = runner.run()

import numpy as np 
mb_act_oh = np.zeros((mb_actions.shape[0], mb_actions.max()+1))
mb_act_oh[np.arange(mb_actions.shape[0]), mb_actions] = 1


expert_file_name = f"../_data/pns_rl_{num_episodes}.npz"
import numpy as np 
np.savez(expert_file_name, obs=mb_obs, acs=mb_act_oh, num_epi = num_episodes, mean_reward = np.sum(mb_rewards)/num_episodes)

Scenic Environment:  ../_scenarios/academy/pass_n_shoot.scenic
in run


In [5]:
np.sum(mb_dones)

499

In [6]:
np.sum(mb_rewards)

406.0

In [7]:
print(mb_obs.shape, mb_actions.shape, mb_act_oh.shape)

(16562, 72, 96, 16) (16562,) (16562, 19)


In [8]:

#expert_file_name = "../_data/pns_100.npz"
#import numpy as np 
#np.savez(expert_file_name, obs=mb_obs, acs=mb_actions)


In [9]:
td = np.load(expert_file_name)

In [10]:
list(td.keys())

['obs', 'acs', 'num_epi', 'mean_reward']

In [11]:
from gfrl.common.mybase.cloning.dataset import GFDset
dataset = GFDset(expert_file_name)

In [12]:
dataset.mean_reward, dataset.size

(array(0.812), 16562)